In [54]:
import os

In [55]:
pwd

'C:\\Users\\dhyey\\Desktop\\Data Science\\Coccidiosis-Classification_Project'

In [13]:
os.chdir("../")

In [14]:
pwd

'C:\\Users\\dhyey\\Desktop\\Data Science\\Coccidiosis-Classification_Project'

In [62]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: str
    tensorboard_root_log_dir: str
    checkpoint_model_filepath: str

In [63]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [64]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAM_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])



    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=str(config.root_dir),
            tensorboard_root_log_dir=str(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=str(config.checkpoint_model_filepath)
        )

        return prepare_callback_config

In [65]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [66]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config



    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)


    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [67]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

except Exception as e:
    raise e

[2023-12-05 14:54:13,971: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-05 14:54:13,971: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-05 14:54:13,971: INFO: common: created directory at: artifacts]
[2023-12-05 14:54:13,971: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2023-12-05 14:54:13,971: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]


In [68]:
import time
timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
f"tb_logs_at_{timestamp}"

'tb_logs_at_2023-12-05-14-54-17'